In [1]:
from pitchtypes import AbstractBase, Spelled, Enharmonic, LogFreq, Converters
from math import log

## Basic Pitch Types

Both _SpelledPitch_ and _EnharmonicPitch_ can be initialised from commonly used pitch notation. However, _SpelledPitch_ will (you guessed it) retain the spelling while _EnharmonicPitch_ converts to the corresponding enharmonically equivalent pitch (internally stored as the corresponding integer in MIDI representation). When printing 'black keys', _EnharmonicPitch_ will need to add an accidental, which by default is a sharp (the 'flat' version can be obtained by explicitly calling the _name_ function with the corresponding argument). _SpelledPitch_ can be converted to _EnharmonicPitch_ but not the other way around.

In [2]:
for pitch in ["F##", "Bbb", "C#", "Db"]:
    print(f"Spelled.PitchClass('{pitch}'):\t\t{Spelled.PitchClass(pitch)}")
    p = Enharmonic.PitchClass(pitch)
    print(f"Enharmonic.PitchClass('{pitch}'):\t{p}\t[{p.name(flat_sharp='sharp')}/{p.name(flat_sharp='flat')}]")
    print(f"{Spelled.PitchClass(pitch)} --> {Spelled.PitchClass(pitch).convert_to(Enharmonic.PitchClass)}")
    # print(f"{Enharmonic.PitchClass(pitch)} --> {Enharmonic.PitchClass(pitch).convert_to(Spelled.PitchClass)}")  # NotImplementedError
    print()

Spelled.PitchClass('F##'):		F##
Enharmonic.PitchClass('F##'):	G	[G/G]
F## --> G

Spelled.PitchClass('Bbb'):		Bbb
Enharmonic.PitchClass('Bbb'):	A	[A/A]
Bbb --> A

Spelled.PitchClass('C#'):		C#
Enharmonic.PitchClass('C#'):	C#	[C#/Db]
C# --> C#

Spelled.PitchClass('Db'):		Db
Enharmonic.PitchClass('Db'):	C#	[C#/Db]
Db --> C#



### Pitch versus Pitch Classe

In fact, the example above is using pitch _classes_, that is the octave is ignored. If you want actual pitches, you will need to specify the octave.

In [3]:
for pitch in ["Dbb1", "C2", "B#3", "C4"]:
    midi = int(Enharmonic.Pitch(pitch))
    print(f"{Spelled.Pitch(pitch)}\t{Enharmonic.Pitch(pitch)}\t(MIDI: {midi} <--> {Enharmonic.Pitch(midi)})")

Dbb1	C1	(MIDI: 24 <--> C1)
C2	C2	(MIDI: 36 <--> C2)
B#3	C4	(MIDI: 60 <--> C4)
C4	C4	(MIDI: 60 <--> C4)


As you can see, _EnharmonicPitch_ can also be converted back and forth from/to the corresponding MIDI integer representation (when providing an integer you have to assist by specifying how it should be interpreted). And if you have a pitch (or interval) object, you can easily convert it to the corresponding pitch (or interval) _class_:

In [4]:
p = Enharmonic.Pitch("C4")
i = Enharmonic.Interval(17)
pc = p.to_class()
ic = i.to_class()
print(f"{p} --> {pc}")
print(f"{i} --> {ic}")

C4 --> C
+17 --> +5


### Frequencies

If you are interested in the frequency corresponding to a specific pitch...well, you should think twice what that means, but here are some ways to deal with frequencies.

We use the _LogFreqPitch_ type to represent frequencies. As the name suggests, it corresponds to the logarithm of the frequency (in Hertz) and is internally represented as a float:

In [5]:
print(LogFreq.Pitch(log(440.1)))
print(LogFreq.Pitch(440.1, is_freq=True))
print(f"{float(LogFreq.Pitch(log(440.1)))} == {log(440.1)}")

440.1Hz
440.1Hz
6.0870019738170456 == 6.0870019738170456


For other pitch types, it is not obvious how they relate to frequency, because this generally depends on the specific tuning being used. To still be as user-friendly as possible, we assume that people using _EnharmonicPitch_ will generally use twelve-tone equal temperament, so we provide a default conversion to frequency:

In [6]:
print(Enharmonic.Pitch("A4").freq())
print(Enharmonic.Pitch("A4").convert_to(LogFreq.Pitch))

440.0
440.Hz


In contrast, _SpelledPitch_ pitch contains additional information that may be used to adjust the frequency in an appropriate way, depending on the tuning and/or the musical context. So we do not provide a default conversion and the following would result in an _NotImplementedError_:

In [7]:
# Spelled.Pitch("A4").convert_to(LogFreq.Pitch)  # NotImplementedError

You can still perform the conversion explicitly, in which case we trust you that you know what you are doing:

In [8]:
Spelled.Pitch("A4").convert_to(Enharmonic.Pitch).convert_to(LogFreq.Pitch)

440.Hz

## Intervals and Arithmetics

ToDo...

In [9]:
p1 = Spelled.Pitch("C#4")
p2 = Spelled.Pitch("Gb5")
i = p2 - p1
print(f"{p2} - {p1} = {i}")
print(f"{p2} + {i} = {p2 + i}")
print(f"{p2} - {i} = {p2 - i}")
print(f"{i} - {i} = {i - i}")
print(f"{i} + {i} = {i + i}")
# p1 + p2                                               # TypeError (cannot add pitches)
# Spelled.PitchClass("G") - Spelled.Pitch("G4")         # TypeError (cannot mix pitches with pitch classes)
# Spelled.PitchClass("G") - Enharmonic.PitchClass("C")  # TypeError (cannot mix different pitch types)

Gb5 - C#4 = +dd5:1
Gb5 + +dd5:1 = Dbbb6
Gb5 - +dd5:1 = C#4
+dd5:1 - +dd5:1 = +P1:0
+dd5:1 + +dd5:1 = +ddd2:2


## Converters

When implementing new pitch types, you can register converters to allow (implicit) conversion to and from existing pitch types.

In [10]:
@AbstractBase.create_subtypes()
class TypeA(AbstractBase): pass

@AbstractBase.create_subtypes()
class TypeB(AbstractBase): pass

@AbstractBase.create_subtypes()
class TypeC(AbstractBase): pass

Converters.register_converter(from_type=TypeA, 
                         to_type=TypeB, 
                         conv_func=lambda pitch_a: TypeB(pitch_a._value, pitch_a.is_pitch, pitch_a.is_class))
Converters.register_converter(from_type=TypeB, 
                         to_type=TypeC, 
                         conv_func=lambda pitch_b: TypeC(pitch_b._value, pitch_b.is_pitch, pitch_b.is_class))

print(TypeA.Pitch("foo").convert_to(TypeB.Pitch))
print(TypeB.Pitch("bar").convert_to(TypeC.Pitch))

TypeBPitch(foo)
TypeCPitch(bar)


### Implicit Converters

In [11]:
# print(TypeA("baz", True, False).convert_to(TypeC))  # NotImplementedError

We did not provide a converter from TypeA to TypeC so attempting a conversion results in an error being raised. We can however request _implicit_ converters to be automatically added by chaining two or more existing converters. This is done for the reverse direction TypeC --> TypeB --> TypeA as follows:

In [12]:
Converters.register_converter(from_type=TypeC,
                         to_type=TypeB, 
                         conv_func=lambda pitch_c: TypeB(pitch_c._value, pitch_c.is_pitch, pitch_c.is_class))
Converters.register_converter(from_type=TypeB,
                         to_type=TypeA, 
                         conv_func=lambda pitch_b: TypeA(pitch_b._value, pitch_b.is_pitch, pitch_b.is_class),
                         create_implicit_converters=True)  # Here, the implicit converter is created!
print(TypeC.Pitch("foo").convert_to(TypeB.Pitch))
print(TypeB.Pitch("bar").convert_to(TypeA.Pitch))
print(TypeC.Pitch("baz").convert_to(TypeA.Pitch))  # no error!

TypeBPitch(foo)
TypeAPitch(bar)
TypeAPitch(baz)


You can look at all registered converters and see how the implicit converter from above is made up from two separate conversion steps arranged into a pipeline. In fact, there are only the two predefined converters we already know (_Spelled_ --> _Enharmonic_, _Enharmonic_ --> _LogFreq_) and those we just defined (including the implicitly defined one):

In [13]:
for from_type, d in Converters._converters.items():
    print(f"from: {from_type}")
    for to_type, f in d.items():
        print(f"    to: {to_type}:\t conversion pipeline: {f}")

from: <class 'pitchtypes.datatypes.Spelled'>
    to: <class 'pitchtypes.datatypes.Enharmonic'>:	 conversion pipeline: [<function <lambda> at 0x105493dc0>]
from: <class 'pitchtypes.datatypes.Enharmonic'>
    to: <class 'pitchtypes.datatypes.LogFreq'>:	 conversion pipeline: [<function <lambda> at 0x106936e50>]
from: <class '__main__.TypeA'>
    to: <class '__main__.TypeB'>:	 conversion pipeline: [<function <lambda> at 0x1054ea0d0>]
from: <class '__main__.TypeB'>
    to: <class '__main__.TypeC'>:	 conversion pipeline: [<function <lambda> at 0x1131948b0>]
    to: <class '__main__.TypeA'>:	 conversion pipeline: [<function <lambda> at 0x1053bc550>]
from: <class '__main__.TypeC'>
    to: <class '__main__.TypeB'>:	 conversion pipeline: [<function <lambda> at 0x1053bc820>]
    to: <class '__main__.TypeA'>:	 conversion pipeline: [<function <lambda> at 0x1053bc820>, <function <lambda> at 0x1053bc550>]


## Type Structure

Any type has four related sub-types:
 * **Pitch:** an actual pitch (like a key on the piano for enharmonic equivalent pitch)
 * **Interval:** the difference between two pitches
 * **PitchClass:** an octave-equivalent pitch (all the "C"s – "C1", "C2", "C3" etc – are pitches that correspond to the same pitch _class_ "C")
 * **IntervalClass:** an octave-equivalent interval (the interval from "C1" to "D1" and that from "C1"to "D2" correspond to the same interval _class_)
 
These sub-types are grouped together by a base class that acts as a conceptual unit and makes the code more readable. So instead of importing all four sub-types separately, you can import the base class and access the sub-types from there:

In [14]:
from pitchtypes import Spelled, SpelledPitch, SpelledInterval, SpelledPitchClass, SpelledIntervalClass
print(SpelledPitch is Spelled.Pitch)
print(SpelledPitchClass is Spelled.PitchClass)
print(SpelledInterval is Spelled.Interval)
print(SpelledIntervalClass is Spelled.IntervalClass)

True
True
True
True


In fact, each of the sub-types is internally derived from that base class to allow for sharing common functionality:

In [15]:
print(issubclass(SpelledPitch, Spelled))
print(issubclass(SpelledPitchClass, Spelled))
print(issubclass(SpelledInterval, Spelled))
print(issubclass(SpelledIntervalClass, Spelled))

True
True
True
True


### Implementing New Types

If you want to implement your own types, we make it easy to retain this clean structure with minimal overhead.

#### All Functionality in the Base Class

If you want to implement all the functionality in the base class you can do this:

In [16]:
from pitchtypes import AbstractBase

@AbstractBase.create_subtypes()
class New(AbstractBase):
    def some_function(self):
        print(self.value)

The decorator `@AbstractBase.create_subtypes()` automatically creates all the sub-types for you:

In [17]:
print(New.Pitch("new"))
print(New.Interval("new"))
print(New.PitchClass("new"))
print(New.IntervalClass("new"))

NewPitch(new)
NewInterval(new)
NewPitchClass(new)
NewIntervalClass(new)


And they share the functionality implemented in the base class:

In [18]:
New.Pitch("some").some_function()
New.Interval("nice").some_function()
New.PitchClass("shared").some_function()
New.IntervalClass("function").some_function()

some
nice
shared
function


In fact, by default they also inherit the common arithmetic operations implemented in `AbstractBase`, unless you override them:

In [19]:
New.Pitch(9) - New.Pitch(7)

NewInterval(2)

#### All Functionality in the Sub Types

If you prefer to implement everything in the sub-types, it is also easy to ensure they all work togeter seamlessly:

In [20]:
class New(AbstractBase):
    pass

@New.link_pitch_type()
class NewPitch(New):
    def f(self):
        print(f"I'm a pitch with value: {self.value}")

@New.link_interval_type()
class NewInterval(New):
    def f(self):
        print(f"I'm an interval with value: {self.value}")

@New.link_pitch_class_type()
class NewPitchClass(New):
    def f(self):
        print(f"I'm a pitch class with value: {self.value}")

@New.link_interval_class_type()
class NewIntervalClass(New):
    def f(self):
        print(f"I'm an interval class with value: {self.value}")

New.Pitch("x").f()
New.Interval("x").f()
New.PitchClass("x").f()
New.IntervalClass("x").f()

I'm a pitch with value: x
I'm an interval with value: x
I'm a pitch class with value: x
I'm an interval class with value: x
